## Notes

Aha: I can convert dataset > pandas to analyze the data.

## Code

In [ ]:
%%capture
!pip install datasets

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [ ]:
def conver2pandas(data):
    data.set_format(type="pandas")
    df = data["train"][:]
    return df


df = conver2pandas(dataset)

df["label"].value_counts(ascending=True).plot.bar()
plt.title("Frequency of Classes")
plt.show()

In [ ]:
n_classes = len(df["label"].value_counts())

In [ ]:
## Retornar formato original
dataset.reset_format()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-cased",
    num_labels=n_classes,
    torch_dtype="auto",

    )

In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_steps=1
)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()